In [154]:
import pandas as pd
from bqplot import ColorScale, ColorAxis, DateScale, LinearScale, Axis, Lines, Figure
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from ipywidgets import Label, VBox, HBox, Layout, Accordion, Dropdown, widgets, interact, Button, Output
from bqplot.market_map import MarketMap
import os
from datetime import datetime
from iexfinance import get_historical_data
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
import warnings
from IPython.display import clear_output
warnings.simplefilter("ignore", DeprecationWarning)

## RETRIEVE AND CLEAN DATA

In [94]:
etf_data = pd.read_excel('../us_etfs.xlsx').drop_duplicates()
etf_data.columns = [i.lower() for i in etf_data.columns]
etf_list = list(etf_data['ticker'])
def get_iex_data(stock_list, start, end):
    return_list = []
    for i in stock_list:
        df = pd.DataFrame(get_historical_data(i, start, end, output_format='pandas')).interpolate()
        df['ticker'] = i
        return_list.append(df)
    return return_list

In [162]:
# full = get_iex_data(etf_list, start=None, end=None)

In [97]:
def lstm_clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
        data[i] = data[i].set_index('date')
        data[i]['Reg_Target'] = data[i]['close'].shift(-1)
    return data

def add_past(etf_list, times):
    for i in range(len(etf_list)):
        for n in times:
            etf_list[i]['{}day return'.format(n)] = -etf_list[i]['close'].diff(periods=n).round(3)
    return etf_list

In [98]:
clean_full = lstm_clean_data(full)
data = add_past(clean_full, [1, 5, 21, 252])
# inp = add_past(clean_full, [1,2,3,4,5])

## RUN MODELS

In [7]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import timeit

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [8]:
def lstm_time_test_split(X, n_past, date):
    X = X.reset_index()
    scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    ticker = X['ticker'].iloc[0]
    x_train = X[X['date'] < date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])
    scaler.fit(x_train)
    x_test = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[:-1]
    x_train = scaler.transform(x_train)
    x_train = np.reshape(x_train,(x_train.shape[0], n_past, x_train.shape[1]))
    x_test = scaler.transform(x_test)
    x_test = np.reshape(x_test,(x_test.shape[0], n_past, x_test.shape[1]))
    y_train = np.array(X[X['date'] < date]['Reg_Target'].drop(columns='date')).ravel().astype('float').reshape(-1,1)
    y_scaler.fit(y_train)
    y_train = y_scaler.transform(y_train)
    y_test = np.array(X[X['date'] >= date]['Reg_Target'].drop(columns='date')).ravel().astype('float')[:-1].reshape(-1,1)
    y_test = y_scaler.transform(y_test)
    x_holdout = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[-1:]
    x_holdout = scaler.transform(x_holdout)
    x_holdout = np.reshape(x_holdout,(x_holdout.shape[0], n_past, x_holdout.shape[1]))
#     y_test = scaler.transform(y_test)
    return ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler

def build_step_model(x_train, y_train, epoc):
    model = Sequential()
    model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('relu'))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(x_train, y_train, epochs=epoc, batch_size=64, validation_split=.07, verbose=2,shuffle=False)
    return model, history

def yield_preds(model, scaler, x_test, x_holdout, y_test):
    yhat = model.predict(x_test)
    preds = scaler.inverse_transform(yhat)
    true = scaler.inverse_transform(y_test)
    today = model.predict(x_holdout)
    today_pred = scaler.inverse_transform(today)
    return preds, today_pred

def run_all_lstms(data, split, epoc):
    out = pd.DataFrame()
    tomorrow = pd.DataFrame()
    start = timeit.default_timer()
    for i in range(len(data)):
        ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler = lstm_time_test_split(data[i], 1, split)
        print('Model #: {}'.format(i))
        model, history = build_step_model(x_train, y_train, epoc)
        preds, future = yield_preds(model, y_scaler, x_test, x_holdout, y_test)
        out[ticker] = preds.flatten()
        tomorrow[ticker] = future.flatten()
    out = out.set_index(data[0][-len(out):].index)
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    return out, tomorrow

## MAKE DASHBOARD INPUTS

In [99]:
def make_yesterday_mapframe(ref):
    yesterday_inputs = pd.DataFrame([ref[i].reset_index().iloc[-1] for i in range(len(ref))])
    mapframe = pd.merge(etf_data, yesterday_inputs, on='ticker').drop(columns = 'Reg_Target')
    return mapframe

In [117]:
mapframe_y = make_yesterday_mapframe(data)
mapframe_y.columns = ['ticker','Fund Name','Issuer','Expense Ratio','Total Assets','spread (%)','Segment','date','Open','High','Low','Previous Close',
                    'volume','1 Day return', '1 Week return', '1 Month return','1 Year return']

In [102]:
def make_daily_mapframe(ref, value):
    mapframe = pd.DataFrame()
    for i in range(len(ref)):
        ticker = ref[i]['ticker'].iloc[0]
        mapframe[ticker] = ref[i][value]
#     mapframe = mapframe.set_index(ref[0]['date']) 
    return mapframe

In [103]:
mapframe_daily_close = make_daily_mapframe(data, 'close')

In [123]:
df = [i for i in full if etf_list in list(i['ticker'])]

In [13]:
mapframe_preds = pd.read_csv('important_backup.csv').set_index('date')

In [126]:
def explore(ticker):
    df = [i for i in full if ticker in list(i['ticker'])][0]
    trace = go.Candlestick(x=df.reset_index()['date'],
                           open=df['open'],
                           high=df['high'],
                           low=df['low'],
                           close=df['close'])
    layout = go.Layout(title='{} Price (USD)'.format(ticker),xaxis = dict(
            rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])),
            rangeslider = dict(
                visible = False)))
    inp = [trace]
    fig = go.Figure(data=inp,layout=layout)
    iplot(fig)
interact(explore, ticker=etf_list)

interactive(children=(Dropdown(description='ticker', options=('SPY', 'IVV', 'VTI', 'VOO', 'VEA', 'QQQ', 'EFA', 'IEFA', 'AGG', 'VWO', 'IJH', 'IWM', 'IEMG', 'IJR', 'VTV', 'IWF', 'IWD', 'BND', 'VUG', 'LQD', 'XLF', 'VNQ', 'VIG', 'GLD', 'EEM', 'BSV', 'TIP', 'VB', 'VO', 'DIA', 'VEU', 'VYM', 'VCSH', 'IVW', 'VGT', 'XLK', 'MDY', 'VCIT', 'XLV', 'XLE', 'IWB', 'IWR', 'EWJ', 'DVY', 'USMV', 'SCHF', 'SDY', 'PFF', 'VGK', 'RSP', 'SHV', 'ITOT', 'IVE', 'EMB', 'SCHX', 'XLY', 'SHY', 'HYG', 'VBR', 'VV', 'SCHB', 'BIV', 'BNDX', 'MBB', 'FLOT', 'XLI', 'VT', 'IXUS', 'MINT', 'IWS', 'VXUS', 'IAU', 'IGSB', 'MTUM', 'AMLP', 'MUB', 'IWO', 'SCZ', 'IWN', 'GDX', 'IEF', 'XLP', 'VOE', 'IBB', 'EFAV', 'IWP', 'EZU', 'VHT', 'IWV', 'ACWI', 'TLT', 'JNK', 'VBK', 'FDN', 'VFH', 'SCHD', 'SCHA', 'BKLN', 'IJK', 'XLU', 'IEI', 'SPLV', 'VMBS', 'QUAL', 'GOVT', 'EWZ', 'SCHG', 'IJT', 'VXF', 'IJS', 'HDV', 'SCHP', 'IJJ', 'ITA', 'IGIB', 'EFV', 'VTIP', 'GUNR', 'DBEF', 'DXJ', 'VOT', 'VNQI', 'SCHZ', 'VSS', 'PRF', 'IUSG', 'XBI', 'SCHM', 'PGX', 'NEAR', 'SLV', 'HEDJ', 'KRE', 'OEF', 'SCHV', 'FXI', 'FNDX', 'GDXJ', 'SPSB', 'EEMV', 'SCHH', 'SCHE', 'IUSV', 'CWB', 'EMLC', 'DGRO', 'FVD', 'IDV', 'FLRN', 'INDA', 'VDE', 'BIL', 'VDC', 'IYW', 'FNDF', 'VPL', 'MGK', 'PCY', 'XLB', 'GSLC', 'VIS', 'SPIB', 'TQQQ', 'SHM', 'SJNK', 'IYR', 'NOBL', 'VTEB', 'EFG', 'EWY', 'SPYG', 'SRLN', 'VLUE', 'AAXJ', 'FPE', 'EWT', 'KBE', 'FNDA', 'SPDW', 'DON', 'HEFA', 'VGSH', 'ACWV', 'FBT', 'SHYG', 'SPAB', 'MCHI', 'TOTL', 'XOP', 'SCHR', 'FEZ', 'SCHO', 'EWG', 'VCR', 'RWX', 'VPU', 'ACWX', 'TFI', 'EWC', 'AMJ', 'FTSM', 'XLC', 'VGIT', 'DBC', 'JPST', 'IGF', 'PDBC', 'SPHD', 'FV', 'IHI', 'HYD', 'SPTM', 'IEUR', 'XLRE', 'IXN', 'RWR', 'VCLT', 'VAW', 'SSO', 'EWH', 'IYH', 'RPG', 'FTEC', 'GBIL', 'IUSB', 'QTEC', 'DGRW', 'XT', 'ISTB', 'IEV', 'EMLP', 'STIP', 'RWO', 'EPP', 'VOOG', 'SDOG', 'USIG', 'BLV', 'SCHC', 'VRP', 'FNDE', 'DES', 'MGV', 'VONG', 'GVI', 'FXL', 'FLGE', 'PRFZ', 'IGV', 'BBJP', 'DLN', 'SPYV', 'BOND', 'SLYG', 'SKYY', 'FTSL', 'IOO', 'ONEQ', 'ICF', 'FNDC', 'BOTZ', 'QLD', 'EWU', 'DEM', 'ROBO', 'QDF', 'IYF', 'DLS', 'USO', 'SUB', 'LMBS', 'SLYV', 'TDTT', 'IXJ', 'TBT', 'HYS', 'GSY', 'FXR', 'VONV', 'ITM', 'HACK', 'IYG', 'SPLG', 'PZA', 'RYT', 'GEM', 'XAR', 'MLPI', 'FAS', 'JPIN', 'MOAT', 'USMC', 'VTWO', 'IXC', 'HEZU', 'PDP', 'FIYY', 'FBGX', 'GNR', 'MGC', 'FHLC', 'PGF', 'FEX', 'IGM', 'SH', 'CWI', 'IPE', 'FDL', 'RSX', 'FIHD', 'XMLV', 'SLQD', 'FNCL', 'GSG', 'SPSM', 'BBCA', 'SPEM', 'NUGT', 'EPI', 'SOXX', 'SMH', 'XSLV', 'FXH', 'PXF', 'BSCK', 'SPHQ', 'BBEU', 'PWV', 'TILT', 'UPRO', 'HYLS', 'PTLC', 'VOX', 'DGS', 'MDYG', 'ILF', 'KWEB', 'REET', 'LRGF', 'PKW', 'EWA', 'IHF', 'ARKK', 'BSCJ', 'DSI', 'OIH', 'FTCS', 'QAI', 'REM', 'IYY', 'PSCH', 'IWY', 'BSJJ', 'FTA', 'EZM', 'EUFN', 'SPTL', 'ANGL', 'SLY', 'FDT', 'GSIE', 'CMF', 'FXO', 'FPX', 'HEWJ', 'AOR', 'CFO', 'BWX', 'VXX', 'PXH', 'EWL', 'EWW', 'SPMD', 'BSCL', 'VWOB', 'DFJ', 'IYE', 'INTF', 'TLTD', 'IWC', 'IYJ', 'VIOO', 'JKE', 'DBEU', 'PPA', 'PHB', 'RDIV', 'RWL', 'DBJP', 'BSJK', 'JKD', 'RPV', 'MLPA', 'SDIV', 'DHS', 'GXC', 'FTC', 'IGE', 'TDTF', 'DJP', 'AIA', 'ITB', 'IYC', 'VONE', 'VYMI', 'MDYV', 'EWP', 'IVOO', 'TDIV', 'AOM', 'ASHR', 'DFE', 'BAB', 'IDEV', 'IAGG', 'FNX', 'XNTK', 'SDS', 'IGOV', 'BSCM', 'AOA', 'IPAC', 'KBWB', 'DWM', 'VIGI', 'RODM', 'VOOV', 'JNUG', 'IBDM', 'SPXL', 'CDC', 'CSM', 'DWX', 'STPZ', 'GWX', 'EWQ', 'MOO', 'PHO', 'CORP', 'IQDF', 'DTN', 'FEP', 'IBDL', 'UYG', 'SGOL', 'IVOG', 'LIT', 'NFRA', 'TNA', 'COMT', 'PEY', 'CIBR', 'KIE', 'IYT', 'XLG', 'SUSA', 'IAT', 'PID', 'USRT', 'TVIX', 'EES', 'MJ', 'FYX', 'RYH', 'SQQQ', 'NANR', 'INDY', 'SPYD', 'JKG', 'PCEF', 'FDIS', 'XHE', 'XME', 'IVOV', 'IFV', 'IBDK', 'GDVD', 'BSCI', 'VNLA', 'VGLT', 'RDVY', 'FFEU', 'MDIV', 'XRT', 'MNA', 'XHB', 'TECL', 'PSK', 'PWB', 'CFA', 'AMZA', 'JHML', 'IDU', 'PTMC', 'DTD', 'USCI', 'ITE', 'IYM', 'SOXL', 'VIDI', 'SNLN', 'BBAX', 'JHMM', 'LVHD', 'KXI', 'IGLB', 'VUSE', 'FLTR', 'RFG', 'FENY', 'BSJI', 'TLH', 'ICSH', 'DBA', 'CGW', 'EBND', 'PNQI', 'JPUS', 'EDV', 'FVC', 'SPTS', 'TBF', 'PJP', 'FREL', 'MLPX', 'EWM', 'HYMB', 'PFXF', 'IGHG', 'ARK

<function __main__.explore>

In [161]:
col = ColorScale()
segments = mapframe_y['Segment'].values
issuers = mapframe_y['Issuer'].values
market_map1 = MarketMap(names=etf_list,      
                       # basic data which needs to set for each map
                       ref_data=mapframe_y, groups=segments, cols=50, row_groups=8,
                       tooltip_fields=['Fund Name', 'Segment', 'Issuer', 'Expense Ratio', 'Total Assets', 'Previous Close', '1 Day return', '1 Week return', '1 Month return', '1 Year return'],
                       scales={'color': col},
                       layout={'min_width': '1500px', 'min_height': '1100px'})
market_map1.font_style = {'font-size': '8px', 'fill':'black'}
market_map1.title = 'ETF Market Map'
out = Output()

drop1 = Dropdown(options=['Groups', '1 Day return', '1 Week return', '1 Month return', '1 Year return'], value= 'Groups', description='Color By:', disabled=False,)
drop2 = Dropdown(options=['Segment', 'Issuer'], value='Segment', description='Group By:', disabled=False,)
button1 = Button(description='Clear Plots')
selectors = HBox([drop1, drop2, button1])


def on_d1_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.color = mapframe_y[change['new']]
        market_map1.axes = [ColorAxis(scale=col, label=change['new'], visible=True)]
        
def on_d2_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.groups = mapframe_y[change['new']]
        
def on_box_change(change):
    with out:
        df = [i for i in full if change['new'][-1] in list(i['ticker'])][-1]
        trace = go.Candlestick(x=df.reset_index()['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])
        layout = go.Layout(title='{} Price (USD)'.format(change['new'][-1]),xaxis = dict(
                rangeselector=dict(
                buttons=list([
                    dict(count=1, label='1m', step='month', stepmode='backward'),
                    dict(count=6, label='6m', step='month', stepmode='backward'),
                    dict(step='all')
                ])),
                rangeslider = dict(visible = False)))
        inp = [trace]
        fig = go.Figure(data=inp,layout=layout)
        iplot(fig)
    
def clear_plots(click):
    out.clear_output()
    
drop1.observe(on_d1_change)
drop2.observe(on_d2_change)
button1.on_click(clear_plots)
market_map1.observe(on_box_change, 'selected')
first_page = VBox([selectors, market_map1, out])
first_page

VBox(children=(HBox(children=(Dropdown(description='Color By:', options=('Groups', '1 Day return', '1 Week return', '1 Month return', '1 Year return'), value='Groups'), Dropdown(description='Group By:', options=('Segment', 'Issuer'), value='Segment'), Button(description='Clear Plots', style=ButtonStyle()))), MarketMap(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], cols=50, font_style={'font-size': '8px', 'fill': 'black'}, groups=array(['Equity: U.S. - Large Cap', 'Equity: U.S. - Large Cap',
       'Equity: U.S. - Total Market', ...,
       'Commodities: Industrial Metals Lead',
       'Inverse Commodities: Broad Market',
       'Leveraged Equity: U.S. - Large Cap Growth'], dtype=object), layout=Layout(min_height='1100px', min_width='1500px'), map_margin={'top': 50, 'right': 50, 'left': 50, 'bottom': 50}, names=array(['SPY', 'IVV', 'VTI', ..., 'LD', 'DDP', 'FRLG'], dtype='<U4'), ref_data=     ticker                                          Fund Name  \
0       SPY                             SPDR S&P 500 ETF Trust   
1       IVV                           iShares Core S&P 500 ETF   
2       VTI                    Vanguard Total Stock Market ETF   
3       VOO                               Vanguard S&P 500 ETF   
4       VEA                Vanguard FTSE Developed Markets ETF   
5       QQQ                                  Invesco QQQ Trust   
6       EFA                              iShares MSCI EAFE ETF   
7      IEFA                         iShares Core MSCI EAFE ETF   
8       AGG               iShares Core U.S. Aggregate Bond ETF   
9       VWO                 Vanguard FTSE Emerging Markets ETF   
10      IJH                       iShares Core S&P Mid-Cap ETF   
11      IWM                           iShares Russell 2000 ETF   
12     IEMG             iShares Core MSCI Emerging Markets ETF   
13      IJR                     iShares Core S&P Small Cap ETF   
14      VTV                                 Vanguard Value ETF   
15      IWF                    iShares Russell 1000 Growth ETF   
16      IWD                     iShares Russell 1000 Value ETF   
17      BND                     Vanguard Total Bond Market ETF   
18      VUG                                Vanguard Growth ETF   
19      LQD  iShares iBoxx USD Investment Grade Corporate B...   
20      XLF                  Financial Select Sector SPDR Fund   
21      VNQ                           Vanguard Real Estate ETF   
22      VIG                 Vanguard Dividend Appreciation ETF   
23      GLD                                    SPDR Gold Trust   
24      EEM                  iShares MSCI Emerging Markets ETF   
25      BSV                       Vanguard Short-Term Bond ETF   
26      TIP                              iShares TIPS Bond ETF   
27       VB                             Vanguard Small-Cap ETF   
28       VO                               Vanguard Mid-Cap ETF   
29      DIA        SPDR Dow Jones Industrial Average ETF Trust   
...     ...                                                ...   
2130   TYNS   Direxion Daily 7-10 Year Treasury Bear 1x Shares   
2131    URR                Market Vectors-Double Long Euro ETN   
2132   SPXV               ProShares S&P 500 Ex-Health Care ETF   
2133   SPXN                ProShares S&P 500 Ex-Financials ETF   
2134   SPMV               Invesco S&P 500 Minimum Variance ETF   
2135   GMFL                 Invesco Multi-Factor Large Cap ETF   
2136   VETS            Pacer Military Times Best Employers ETF   
2137   PSMM  Invesco Moderately Conservative Multi-Asset Al...   
2138   PSMC          Invesco Growth Multi-Asset Allocation ETF   
2139    SCC             ProShares UltraShort Consumer Services   
2140   CNHX         CSOP MSCI China A International Hedged ETF   
2141   CHEP               AGFiQ U.S. Market Neutral Value Fund   
2142   HAUD         iShares Currency Hedged MSCI Australia ETF   
2143   USOD                    United States 3x Short Oil 